# numpy
*Think like a vector*
## Example 1: Simple integration
### Goal

Integrate a the function $\sin(x) + x$ between $0$ and $\pi$.

The should be close to $\frac{\pi^2}{2} + 2 = 6.9348$.

### Strategy

We use the standard tTrapezoidal rule
$$
\int_a^b f(x) dx \approx \sum_{j = 1}^n \frac{1}{2} [f(x_{j - 1}) + f(x_j)] (x_j - x_{j-1})
$$
with $x_j = a + j (b - a)/n$.

1. Generate an *array* of $x_j$.

In [3]:
import numpy as np  # We need the array/maths library

xx = np.arange(0, np.pi, 0.001)  # Choose the step
xx = np.linspace(0, np.pi, 1000)  # Choose the number of points

2. Compute the trapeze area *elementwise*.

In [12]:
yy = np.sin(xx) + xx  # All operation are done elmentwise

# Subarray with yy[begin:end]
# We want yy[begin:end - 1] + yy[begin + 1:end]
trapezes = 1/2 * (yy[:-1] + yy[1:]) * (xx[1:] - xx[:-1])

3. Sum over the array, *reducing* it to single number.

In [14]:
res = np.sum(trapezes)
print(res)

6.93480055231553


## Example 2: Ising model
### Goal

Create a box with $L$ layers, each a $N \times N$ grid of random spins.

The probability of the spins being in the up state in each layer is stored in the file `layer_magnetizations.txt`.

Compute the average spin per layer.

Compute the total energy, with couplings $J_x = 1$, $J_y = 1.2$ and $J_z = 0.5$.

### Strategy
1. Create the box with all spins down.

In [5]:
N = 20
L = 10

spins = -np.ones((L, N, N), dtype = int)  # Normally you don't care about dtype
print(spins.shape)
print(spins[1, 1, 1])

(10, 20, 20)
-1


2. Load the magnetizations from file.

In [13]:
up_probabilities = np.loadtxt("../magnetizations.txt")

print(up_probabilities)
print(up_probabilities.shape)  # Check the size

[1.         0.5        0.33333333 0.25       0.2        0.16666667
 0.14285714 0.125      0.11111111 0.1       ]
(10,)


3. Switch the spins according to the given probability.

In [32]:
rr = np.random.rand(L, N, N)

# ups = rr < up_probabilities  is a broadcast error
print(up_probabilities[:, None, None].shape)  # Extend it
ups = rr < up_probabilities[:, None, None]

# mask indexing
spins[ups] = 1


[1.         0.5        0.33333333 0.25       0.2        0.16666667
 0.14285714 0.125      0.11111111 0.1       ]
(10,)
(10, 1, 1)


4. Compute the average spin per layer.

In [34]:
avg_spins = np.mean(spins, axis=(1, 2))
print(avg_spins)

[ 1.    -0.065 -0.35  -0.49  -0.615 -0.665 -0.72  -0.775 -0.8   -0.8  ]


5. Compute the total energy from each site energy.

$$
E_{ijk} = J_x S_{ijk} S_{(i + 1)jk} + J_y S_{ijk} S_{i(j + 1)k} + J_z S_{ijk} S_{ij(k + 1)}.
$$

In [40]:
Jx = 1
Jy = 1.2
Jz = 0.5

energies = np.zeros(spins.shape)

energies[:-1, :, :] += Jx * spins[:-1, :, :] * spins[1:, :, :]
energies[:, :-1, :] += Jy * spins[:, :-1, :] * spins[:, 1:, :]
energies[:, :, :-1] += Jz * spins[:, :, 1:] * spins[:, :, 1:]

Etot = np.sum(energies)
print(Etot)

5170.4
